In [3]:

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

import numpy as np
import pickle

import os
import sys

module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from tqdm import tqdm
from utils import *

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')



C:\Users\Ignatius Ting\AppData\Local\Temp\ipykernel_28032\405212744.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)
[nltk_data] Downloading package stopwords to C:\Users\Ignatius
[nltk_data]     Ting\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Ignatius
[nltk_data]     Ting\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
with open('../data/counts_1m.pkl', 'rb') as f:
    counts_1m = pickle.load(f)


### Remove unnecessary columns and most frequent concept

In [5]:
rm_cols = ['mapped from',
'mapped to',
'do not code with',
'associated with',
'interprets',
'is interpreted by',
'interpretation of',
'has interpretation'
'other mapped from',
'other mapped to',
'inactive ingredient of',
'has inactive ingredient'
]
counts_1m = counts_1m[~counts_1m.rel.isin(rm_cols)]
counts_1m = counts_1m[(counts_1m.source_text != 'EXCISION') & (counts_1m.cui_2_text != 'EXCISION')].reset_index(drop = True)

In [6]:

import re

EN_STOPWORDS = list(set(stopwords.words('english')))

def count_non_stop_words(text):
    # Tokenize the text
    text = re.sub(r'[^\w\s]', '', text)
    words = word_tokenize(text)
    
    # Get the list of English stop words
    stop_words = EN_STOPWORDS
    
    # Filter out the stop words and count non-stop words
    filtered_words = [word.lower() for word in words if word.lower() not in stop_words]
    return len(filtered_words)
    # return filtered_words

def remove_stopwords(text):
    stop_words = EN_STOPWORDS
    text = re.sub(r'[^\w\s]', '', text)
    words = word_tokenize(text)
    filtered_words = [word.lower() for word in words if word.lower() not in stop_words]
    return set(filtered_words)

### Count number of non-stop words

In [7]:

counts_1m['source_true_len'] = counts_1m.source_text.apply(lambda x: count_non_stop_words(x))
counts_1m['cui_2_true_len'] = counts_1m.cui_2_text.apply(lambda x: count_non_stop_words(x))
# filter for only 5 or less
# counts_1m = counts_1m[(counts_1m.source_true_len <= 5) & (counts_1m.cui_2_true_len <= 5)]

# Apply the function to each row in both columns
counts_1m['source_text_words'] = counts_1m['source_text'].apply(remove_stopwords)
counts_1m['cui_2_text_words'] = counts_1m['cui_2_text'].apply(remove_stopwords)

# Calculate the number of overlapping words
counts_1m['overlap_count'] = counts_1m.apply(lambda row: len(row['source_text_words'].intersection(row['cui_2_text_words'])), axis=1)


### Count overlapping substring

In [ ]:


def largest_overlapping_substring(str1, str2):
    # some preprocessing
    str1 = str1.lower()
    str2 = str2.lower()
    str1 = re.sub(r'[^\w\s]', '', str1)
    str2 = re.sub(r'[^\w\s]', '', str2)
    # Initialize a matrix to store the lengths of overlapping substrings
    matrix = [[0] * (len(str2) + 1) for _ in range(len(str1) + 1)]

    max_length = 0  # Maximum overlapping substring length
    end_index = 0   # Ending index of the maximum overlapping substring
    
    for i in range(1, len(str1) + 1):
        for j in range(1, len(str2) + 1):
            if str1[i - 1] == str2[j - 1]:
                matrix[i][j] = matrix[i - 1][j - 1] + 1
                if matrix[i][j] > max_length:
                    max_length = matrix[i][j]
                    end_index = i  # Update the ending index
    
    # Extract the largest overlapping substring
    largest_overlap = str1[end_index - max_length:end_index]
    
    return largest_overlap



In [ ]:
# Apply the function to each row in the DataFrame
counts_1m['largest_overlap'] = counts_1m.apply(lambda row: largest_overlapping_substring(row['source_text'], row['cui_2_text']), axis=1)

# Compare the length of the largest overlapping substring to half the length of the shorter string
counts_1m['overlap_pass'] = counts_1m.apply(lambda row: len(row['largest_overlap']) > min(len(row['source_text']), len(row['cui_2_text'])) / 2, axis=1)


In [ ]:
counts_1m[['source_text']].sample(3)

,source_text
688610,puncture of left wrist with surrounding tenderness
239701,"Left lobe of liver, NOS"
752436,FORSYTHE-WAKELING SYNDROME


In [25]:
counts_1m = counts_1m[(counts_1m.source_true_len <= 5) & (counts_1m.cui_2_true_len <= 5)]
counts_1m = counts_1m[counts_1m.overlap_count < 3]
counts_1m = counts_1m[counts_1m.overlap_pass == False]

In [26]:
counts_1m.reset_index(drop = True, inplace = True)

In [27]:
with open('../data/counts_1m_trim.pkl', 'wb') as file:
    pickle.dump(counts_1m, file)